In [2]:
import lightgbm as lgb 
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np 
import pandas as pd 
import os 
import sys 
sys.path.append('src/')
from feature_generation import feature_generation 
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score 
import xgboost as xgb
from sklearn.metrics import classification_report

In [23]:
train_data = np.load('训练集/train_x.npy')
train_label = np.load('训练集/train_y.npy')
test_data = np.load('测试集A/test_x_A.npy')
sample_submit = pd.read_csv('测试集A/submit_example_A.csv', index_col=0)

In [24]:
train_fg = feature_generation(train_data)
train_fg.get_feature()
train_fg.feature_concat()
train_data = train_fg.X
train_feature_name = train_fg.feature_name
print(train_data.shape)

test_fg = feature_generation(test_data)
test_fg.get_feature()
test_fg.feature_concat()
test_data = test_fg.X
test_feature_name = test_fg.feature_name
print(test_data.shape)

(37549, 910)
(1155, 910)


In [1]:
from model_train_eval import lgb_cv_eval

lgb_params = {'learning_rate': 0.1, 'max_depth': -1, 'min_child_weight': 1,
            'colsample_bytree': 1, 'subsample': 1, 'reg_lambda': 0.5, 'reg_alpha': 0.5,'num_leaves':31,
            'seed': 33,'verbose':1,  'objective':'multiclass' , 'num_class': 3}
num_boost_round = 200
cv_model, dev_df, train_df = lgb_cv_eval(k =10, train_data = train_data, train_label = train_label, 
                                         lgb_params = lgb_params, 
                                         num_boost_round = num_boost_round)

ModuleNotFoundError: No module named 'model_train_eval'

In [8]:
all_prob = []
all_label = []
for i in cv_model:
    model = cv_model[i] 
    pred_prob = model.predict(test_data)
    pred_label = np.argmax(pred_prob, axis = 1)
    for _ in range(3):
        print(f'{_}: {(pred_label == _).sum()}')   
    print('===='* 10) 
    all_prob.append(pred_prob)
    all_label.append(pred_label)

0: 816
1: 221
2: 118
0: 818
1: 197
2: 140
0: 814
1: 206
2: 135
0: 808
1: 205
2: 142
0: 809
1: 201
2: 145
0: 815
1: 194
2: 146
0: 810
1: 205
2: 140
0: 815
1: 211
2: 129
0: 819
1: 206
2: 130
0: 815
1: 198
2: 142


In [18]:
prob = 0
for _ in all_prob:
    prob += _ 

prob = prob/10
pred_label = np.argmax(pred_prob, axis = 1)
for _ in range(3):
    print(f'{_}: {(pred_label == _).sum()}')

0: 815
1: 198
2: 142


In [21]:
pred_label = np.array([2]*test_data.shape[0])

top_385_indices = prob[:,0].argsort()[::-1][:385]
pred_label[top_385_indices] = 0

tmp_res = prob.copy()
tmp_res[top_385_indices,1] = 0

top_385_indices = tmp_res[:,1].argsort()[::-1][:385]
pred_label[top_385_indices] = 1

for _ in np.unique(pred_label):
    print(f'{_}: {(pred_label == _).sum()}')

sample_submit.loc[:,'label'] = pred_label 
sample_submit.to_csv('res_6.csv')

0: 385
1: 385
2: 385
